In [1]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import h5py
import numpy as np
import scipy.io
from os import listdir
from os import mkdir
import pyarrow

# Indicando as pastas de fonte dos arquivos originais da região 5 e destino dos arquivos chaves

In [13]:
"""Pasta de origem dos dados e pasta de destino para os dados trabalhados"""
path_sources = r'C:\Users\vinic\python_projects\monitoracao_project\dados'
path_destiny = r'C:\Users\vinic\python_projects\monitoracao_project\dados_parquet'

folders = listdir(path_sources) # Lembrar de tirar o [0] quando for rodar tudo
#folders = [folders[1]] # teste com uma pasta só
print(folders)

['LRV4306_accelerometer_data', 'LRV4306_accelerometer_data_teste', 'LRV4306_gps_data', 'LRV4306_gps_data_teste', 'LRV4313_accelerometer_data', 'LRV4313_gps_data']


# Código para criação do dataframe das chaves de acesso 

In [10]:
for i in range(len(folders)):

    if 'accel' in folders[i]:
        Chaves = pd.DataFrame(columns=['Data', 'N Passagens', 'Direcao', 'Sensor', 'Chave'])

    elif 'gps' in folders[i]:
        Chaves = pd.DataFrame(columns=['Data', 'N Passagens', 'Direcao', 'Chave'])
    
    name = folders[i]

    path_sour = path_sources+'\\'+folders[i] # entra na pasta com arquivos
    path_dest = path_destiny+'\\'+folders[i] # entra na pasta onde os arquivos serão salvos

    try:
        mkdir(path_dest) # cria pastas de acordo com o nome da pasta de origem
    except:
        pass
    
    files = listdir(path_sour) # Carrega os nomes dos arquivos dentro da pasta de origem

        # Quantidade de linhas no arquivo?


    for n in range(len(files)):
    
        j = files[n]

        props = j.split('_')
        props[-1] = props[-1][0]

        l = path_sour+'\\'+j
        m = path_dest+'\\'+j

        if 'accel' in folders[i]:
            #{file#}_{date}_{root index}_{daily passing#}_{region#}_{running direction}_{sensor channel}.mat
            df_temp = pd.DataFrame({'Data':[props[1]], 'N Passagens':[props[3]], 'Direcao':[props[5]], 'Sensor':[props[6]], 'Chave': l})
            Chaves = pd.concat([Chaves, df_temp], axis=0, ignore_index=True)


        elif 'gps' in folders[i]:

            df_temp = pd.DataFrame({'Data':[props[1]], 'N Passagens':[props[3]], 'Direcao':[props[5]], 'Chave':l})
            Chaves = pd.concat([Chaves, df_temp], axis=0, ignore_index=True)


    Chaves.to_parquet(path_dest+'\\'+name+'_key', engine = 'pyarrow', compression= 'brotli')

# Carregamento de arquivos específicos

In [22]:
Chaves
datas = set(Chaves.loc[:,'Data'])
datas

{'20150618',
 '20150625',
 '20150818',
 '20150924',
 '20151022',
 '20151029',
 '20151105',
 '20151203',
 '20151210',
 '20160114',
 '20160128',
 '20160228',
 '20160407',
 '20160414',
 '20160421',
 '20160505',
 '20160519'}

In [23]:
errors = [] # Guarda qualquer arquivo que falhou em ser carregado
linha = 0
for i in range(len(folders)):

    path_sour = path_destiny+'\\'+folders[i] # entra na pasta com as chaves
    
    files = listdir(path_sour) # Carrega os nomes dos arquivos dentro da pasta de origem

    for n in range(len(files)):
        if 'key' in files[i]:

            l = path_sour+'\\'+files[i]
            
            Chaves = pd.read_parquet(l)
            
            datas = set(Chaves.loc[:,'Data'])
            n_passagens = set(Chaves.loc[:,'N Passagens'])
            direcoes = set(Chaves.loc[:,'Direcao'])

            if 'accel' in files[n]:
                sensores = set(Chaves.loc[:,'Sensor'])
            elif 'gps' in files[n]:
                sensores = None
            


## Teste sensor 1 dados

In [41]:
# tratar dados que são None
print('Teste de qualidade dos dados')
print('--------------------------------')
for i in Chaves.keys():
    print(f'Elementos faltantes na coluna {i}?')
    print(set(Chaves.loc[:, 'Sensor'] == None))
    print(f'Tamanho na coluna {i}')
    print(len(Chaves.loc[:, 'Sensor']))
    print('-----------------------------------')

Teste de qualidade dos dados
--------------------------------
Elementos faltantes na coluna Data?
{False}
Tamanho na coluna Data
3469
-----------------------------------
Elementos faltantes na coluna N Passagens?
{False}
Tamanho na coluna N Passagens
3469
-----------------------------------
Elementos faltantes na coluna Direcao?
{False}
Tamanho na coluna Direcao
3469
-----------------------------------
Elementos faltantes na coluna Sensor?
{False}
Tamanho na coluna Sensor
3469
-----------------------------------
Elementos faltantes na coluna Chave?
{False}
Tamanho na coluna Chave
3469
-----------------------------------
